# Documentation Writing Crew

This notebook demonstrates how to create an AI crew for writing documentation using CrewAI Flows.
The crew will analyze code from any public GitHub repository and generate comprehensive documentation
by working collaboratively using specialized agents with different roles and responsibilities.
CrewAI Flows enable coordinated execution and communication between agents to produce high-quality
documentation for any codebase.

### Initialization and Setup
Initial imports for the CrewAI Flow and Crew and setting up the environment

In [1]:
# Importing necessary libraries
import getpass
import os
import yaml
import subprocess
from pathlib import Path
from pydantic import BaseModel

# Importing Crew related components
from crewai import Agent, Task, Crew

# Importing CrewAI Flow related components
from crewai.flow.flow import Flow, listen, start

# Apply a patch to allow nested asyncio loops in Jupyter
import nest_asyncio
nest_asyncio.apply()

# Setting up the environment
os.environ["NVIDIA_NIM_API_KEY"] = getpass.getpass("Enter your NVIDIA NIM API key: ")
os.environ["NVIDIA_API_KEY"] = getpass.getpass("Enter your NVIDIA API key: ")


# Define the project URL

In [2]:
project_url = "https://github.com/langchain-ai/langchain-nvidia"

## Plan for our Flow

1. Clone the repository for the project
2. Plan the documentation for the project **[Crew of Agents]** 
3. Create the documentation for the project **[Crew of Agents]**

# ![CrewAIFlow.png](crewai-nim-flow.png)

# Optmizing for Llama 3.3 Prompting Template

In [3]:
# Agents Prompting Template for Llama 3.3
system_template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>{{ .System }}<|eot_id|>"""
prompt_template="""<|start_header_id|>user<|end_header_id|>{{ .Prompt }}<|eot_id|>"""
response_template="""<|start_header_id|>assistant<|end_header_id|>{{ .Response }}<|eot_id|>"""

# Create Planning Crew

Initial strucutre data we will use to capture the output of the planning crew

In [4]:
# Define data structures to capture documentation planning output
class DocItem(BaseModel):
    """Represents a documentation item"""
    title: str
    description: str
    prerequisites: str
    examples: list[str]
    goal: str

class DocPlan(BaseModel):
    """Documentation plan"""
    overview: str
    docs: list[DocItem]

In [5]:
from crewai_tools import (
    DirectoryReadTool,
    FileReadTool,
)

# Load agent and task configurations from YAML files
with open('config/planner_agents.yaml', 'r') as f:
    agents_config = yaml.safe_load(f)

with open('config/planner_tasks.yaml', 'r') as f:
    tasks_config = yaml.safe_load(f)

code_explorer = Agent(
  config=agents_config['code_explorer'],
  system_template=system_template,
  prompt_template=prompt_template,
  response_template=response_template,
  tools=[
    DirectoryReadTool(),
    FileReadTool()
  ]
)
documentation_planner = Agent(
  config=agents_config['documentation_planner'],
  system_template=system_template,
  prompt_template=prompt_template,
  response_template=response_template,
  tools=[
    DirectoryReadTool(),
    FileReadTool()
  ]
)

analyze_codebase = Task(
  config=tasks_config['analyze_codebase'],
  agent=code_explorer
)
create_documentation_plan = Task(
  config=tasks_config['create_documentation_plan'],
  agent=documentation_planner,
  output_pydantic=DocPlan
)

planning_crew = Crew(
    agents=[code_explorer, documentation_planner],
    tasks=[analyze_codebase, create_documentation_plan],
    verbose=False
)

/Users/joaomoura/.pyenv/versions/3.11.7/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from typing import Type
from crewai_tools import BaseTool
from pydantic import BaseModel, Field

class ValidateMermaidInput(BaseModel):
    """Input schema for ValidateMermaidTool."""
    text: str = Field(..., description="Mermaid diagram text to validate.")

class ValidateMermaidTool(BaseTool):
    name: str = "Validate Mermaid Diagram"
    description: str = (
        "Validates Mermaid diagram syntax, specifically checking for proper arrow syntax "
        "and ensuring there are no invalid trailing '>' characters or malformed pipe ('|') decorators."
    )
    args_schema: Type[BaseModel] = ValidateMermaidInput

    def _run(self, text: str) -> bool:
        # Check for invalid trailing > in arrow syntax
        if "|>" in text:
            return False

        # Check for valid arrow syntax
        valid_arrows = ["-->", "---", "---|", "|---|"]
        lines = text.split("\n")
        for line in lines:
            if any(arrow in line for arrow in valid_arrows):
                # Verify arrow syntax is properly formatted
                parts = line.split()
                for part in parts:
                    if part.startswith("|") and not part.endswith("|"):
                        return False
                    if part.endswith(">") and not part.startswith("-"):
                        return False

        return True

# Create Documentation Crew

Crew of AI Agents to execute the documentation plan and create the documentation

In [7]:

from crewai_tools import (
    DirectoryReadTool,
    FileReadTool,
    WebsiteSearchTool
)

# Load agent and task configurations from YAML files
with open('config/documentation_agents.yaml', 'r') as f:
    agents_config = yaml.safe_load(f)

with open('config/documentation_tasks.yaml', 'r') as f:
    tasks_config = yaml.safe_load(f)

overview_writer = Agent(
  config=agents_config['overview_writer'],
  system_template=system_template,
  prompt_template=prompt_template,
  response_template=response_template,
  tools=[
    DirectoryReadTool(),
    FileReadTool(),
    ValidateMermaidTool(),
    WebsiteSearchTool(
      website="https://mermaid.js.org/syntax/examples.html",
      config=dict(
        embedder=dict(
            provider="nvidia",
            config=dict(
                model="nvidia/nv-embedqa-e5-v5"
            ),
        ),
      )
    )
])

documentation_reviewer = Agent(
  config=agents_config['documentation_reviewer'],
  system_template=system_template,
  prompt_template=prompt_template,
  response_template=response_template,
  tools=[
    DirectoryReadTool(directory="docs/", name="Check existing documentation folder"),
    ValidateMermaidTool(),
    FileReadTool(),
  ]
)

draft_documentation = Task(
  config=tasks_config['draft_documentation'],
  agent=overview_writer
)

qa_review_documentation = Task(
  config=tasks_config['qa_review_documentation'],
  agent=documentation_reviewer
)

documentation_crew = Crew(
    agents=[overview_writer, documentation_reviewer],
    tasks=[draft_documentation, qa_review_documentation],
    verbose=False
)

2024-12-18 03:54:07,280 - 8071896128 - __init__.py-__init__:521 - WARNING: Overriding of current TracerProvider is not allowed


# Create Documentation Flow

A Flow to create the documentation for the project where we will use the planning crew to plan the documentation and the documentation crew to create the documentation

In [8]:
class DocumentationState(BaseModel):
  """
  State for the documentation flow
  """
  project_url: str = project_url
  repo_path: Path = "workdir/"

class CreateDocumentationFlow(Flow[DocumentationState]):
  # Clone the repository, initial step
  # No need for AI Agents on this step, so we just use regular Python code
  @start()
  def clone_repo(self):
    print(f"# Cloning repository: {self.state.project_url}")
    # Extract repo name from URL
    repo_name = self.state.project_url.split("/")[-1]
    self.state.repo_path = f"{self.state.repo_path}{repo_name}"

    # Clone the repository
    subprocess.run(["git", "clone", self.state.project_url, self.state.repo_path])
    return self.state

  @listen(clone_repo)
  def plan_docs(self):
    print(f"# Planning documentation for: {self.state.repo_path}")
    result = planning_crew.kickoff(inputs={'repo_path': self.state.repo_path})
    print(f"# Planned docs for {self.state.repo_path}:")
    for doc in result.pydantic.docs:
        print(f"    - {doc.title}")
    return result

  @listen(plan_docs)
  def save_plan(self, plan):
    with open("docs/plan.json", "w") as f:
      f.write(plan.raw)

  @listen(plan_docs)
  def create_docs(self, plan):
    for doc in plan.pydantic.docs:
      print(f"# Creating documentation for: {doc.title}")
      result = documentation_crew.kickoff(inputs={
        'repo_path': self.state.repo_path,
        'title': doc.title,
        'overview': plan.pydantic.overview,
        'description': doc.description,
        'prerequisites': doc.prerequisites,
        'examples': doc.examples,
        'goal': doc.goal
      })

      # Save documentation to file in docs folder
      docs_dir = Path("docs")
      docs_dir.mkdir(exist_ok=True)
      title = doc.title.lower().replace(" ", "_") + ".mdx"
      with open(docs_dir / title, "w") as f:
          f.write(result.raw)
    print(f"# Documentation created for: {self.state.repo_path}")


Implementing helper methods to plot and execute the flow in a Jupyter notebook

In [9]:
# Plot the flow
flow = CreateDocumentationFlow()
flow.plot()

# Display the flow visualization using IFrame
from IPython.display import IFrame

# Display the flow visualization
IFrame(src='./crewai_flow.html', width='100%', height=400)

Plot saved as crewai_flow.html


In [10]:
flow = CreateDocumentationFlow()
flow.kickoff()

# Cloning repository: https://github.com/langchain-ai/langchain-nvidia
# Planning documentation for: workdir/langchain-nvidia


fatal: destination path 'workdir/langchain-nvidia' already exists and is not an empty directory.


# Planned docs for workdir/langchain-nvidia:
    - Introduction to LangChain NVIDIA
    - Getting Started with LangChain NVIDIA
    - Architecture and Design of LangChain NVIDIA
    - Using LangChain NVIDIA for AI Applications
    - Advanced Topics in LangChain NVIDIA
# Creating documentation for: Introduction to LangChain NVIDIA
# Creating documentation for: Getting Started with LangChain NVIDIA
# Creating documentation for: Architecture and Design of LangChain NVIDIA
# Creating documentation for: Using LangChain NVIDIA for AI Applications
# Creating documentation for: Advanced Topics in LangChain NVIDIA
# Documentation created for: workdir/langchain-nvidia
